In [ ]:
import os
import shutil
import sys
from google.colab import drive

drive.mount('/content/drive')

if not os.path.exists('/content/models'):
    !git clone https://github.com/Gabrysse/MLDL2024_project1.git temp_repo
    shutil.copytree('temp_repo/models', '/content/models')
    shutil.rmtree('temp_repo')

zip_path = '/content/drive/MyDrive/semseg/project_data.zip'

if not os.path.exists('/content/dataset/project_data/gta5'):
    if not os.path.exists('/content/dataset'): os.makedirs('/content/dataset')

    if os.path.exists(zip_path):
        shutil.unpack_archive(zip_path, '/content/dataset')
    else:
        print(f"Error: Zip file not found at {zip_path}")

Mounted at /content/drive
Cloning into 'temp_repo'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
Receiving objects: 100% (34/34), 11.29 KiB | 11.29 MiB/s, done.
Resolving deltas: 100% (9/9), done.
remote: Total 34 (delta 9), reused 3 (delta 3), pack-reused 13 (from 1)


In [ ]:
import os
import shutil
import sys
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import random
from google.colab import drive

# --- 1. SETUP & IMPORTS ---
print("Setting up environment...")
drive.mount('/content/drive')

# Restore Model Code
if not os.path.exists('/content/models'):
    print("Model code missing. Restoring...")
    !git clone https://github.com/Gabrysse/MLDL2024_project1.git temp_repo
    shutil.copytree('temp_repo/models', '/content/models')
    shutil.rmtree('temp_repo')

from models.bisenet.build_bisenet import BiSeNet

# --- 2. CONFIGURATION ---
CHECKPOINT_NAME = 'bisenet_aug1_checkpoint.pth'
EPOCHS = 50
BATCH_SIZE = 8
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DATASET_DIR = '/content/dataset'
GTA_PATH = os.path.join(DATASET_DIR, 'project_data', 'gta5')
ZIP_PATH = '/content/drive/MyDrive/semseg/project_data.zip'

# --- 3. DATASET CHECK ---
if not os.path.exists(GTA_PATH):
    print("Dataset not found in /content/dataset. Checking zip...")
    if not os.path.exists(DATASET_DIR): os.makedirs(DATASET_DIR)

    if os.path.exists(ZIP_PATH):
        print("Unzipping dataset...")
        shutil.unpack_archive(ZIP_PATH, DATASET_DIR)
        print("Dataset extracted.")
    else:
        print(f"Critical Error: Zip file not found at {ZIP_PATH}")
        sys.exit()
else:
    print("Dataset ready.")

# --- 4. DATASET CLASS (AUG 1 ONLY) ---
class GTA5Aug1Dataset(Dataset):
    def __init__(self, root_dir):
        self.images_dir = os.path.join(root_dir, 'images')
        self.masks_dir = os.path.join(root_dir, 'labels')
        self.images = sorted(os.listdir(self.images_dir))
        self.normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.id_mapping = {
            7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
            19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
            26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
        }

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.images[idx])
        mask_path = os.path.join(self.masks_dir, self.images[idx])

        image = Image.open(img_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        # AUG 1 ONLY: Random Flip
        if random.random() > 0.5:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
            mask = mask.transpose(Image.FLIP_LEFT_RIGHT)

        image = self.normalize(image)
        mask_np = np.array(mask)
        target_mask = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in self.id_mapping.items(): target_mask[mask_np == k] = v
        return image, torch.from_numpy(target_mask).long()

# --- 5. TRAINING LOOP ---
print(f"Starting Training for {CHECKPOINT_NAME}...")
print(f"Batch Size: {BATCH_SIZE} | Device: {DEVICE}")

model = BiSeNet(num_classes=19, context_path='resnet18').to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss(ignore_index=255)

dataset = GTA5Aug1Dataset(GTA_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
save_path = f'/content/drive/MyDrive/{CHECKPOINT_NAME}'

for epoch in range(EPOCHS):
    model.train()
    for i, (img, lbl) in enumerate(loader):
        optimizer.zero_grad()
        out = model(img.to(DEVICE))
        loss = criterion(out[0], lbl.to(DEVICE)) + 0.1 * criterion(out[1], lbl.to(DEVICE)) + 0.1 * criterion(out[2], lbl.to(DEVICE))
        loss.backward()
        optimizer.step()


        if i % 50 == 0:
            print(f"Epoch [{epoch+1}/{EPOCHS}] Step [{i}/{len(loader)}] Loss: {loss.item():.4f}")

    # Save EVERY epoch
    torch.save({'model_state_dict': model.state_dict()}, save_path)
    print(f"Epoch {epoch+1} Saved.")

Setting up environment...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset ready.
Starting Training for bisenet_aug1_checkpoint.pth...
Batch Size: 8 | Device: cuda
Epoch [1/50] Step [0/313] Loss: 3.9578
Epoch [1/50] Step [50/313] Loss: 1.1189
Epoch [1/50] Step [100/313] Loss: 0.6854
Epoch [1/50] Step [150/313] Loss: 0.6066
Epoch [1/50] Step [200/313] Loss: 0.5280
Epoch [1/50] Step [250/313] Loss: 0.5274
Epoch [1/50] Step [300/313] Loss: 0.4635
Epoch 1 Saved.
Epoch [2/50] Step [0/313] Loss: 0.5184
Epoch [2/50] Step [50/313] Loss: 0.5624
Epoch [2/50] Step [100/313] Loss: 0.3988
Epoch [2/50] Step [150/313] Loss: 0.5057
Epoch [2/50] Step [200/313] Loss: 0.4064
Epoch [2/50] Step [250/313] Loss: 0.3651
Epoch [2/50] Step [300/313] Loss: 0.4099
Epoch 2 Saved.
Epoch [3/50] Step [0/313] Loss: 0.3623
Epoch [3/50] Step [50/313] Loss: 0.3850
Epoch [3/50] Step [100/313] Loss: 0.3942
Epoch [3/50] Step [150/313] Lo

In [ ]:
import os
import shutil
import sys
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import random
from google.colab import drive

# --- 1. SETUP & IMPORTS ---
print("Setting up environment...")
drive.mount('/content/drive')

if not os.path.exists('/content/models'):
    print("Restoring Code...")
    !git clone https://github.com/Gabrysse/MLDL2024_project1.git temp_repo
    shutil.copytree('temp_repo/models', '/content/models')
    shutil.rmtree('temp_repo')

from models.bisenet.build_bisenet import BiSeNet

# --- 2. CONFIGURATION ---
CHECKPOINT_NAME = 'bisenet_aug1_checkpoint.pth'
TARGET_EPOCHS = 50
FORCED_START_EPOCH = 48
BATCH_SIZE = 8
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DATASET_DIR = '/content/dataset'
GTA_PATH = os.path.join(DATASET_DIR, 'project_data', 'gta5')
ZIP_PATH = '/content/drive/MyDrive/semseg/project_data.zip'

# --- 3. DATASET CHECK ---
if not os.path.exists(GTA_PATH):
    print("Restoring Dataset...")
    if not os.path.exists(DATASET_DIR): os.makedirs(DATASET_DIR)
    if os.path.exists(ZIP_PATH):
        shutil.unpack_archive(ZIP_PATH, DATASET_DIR)
        print("Dataset ready.")
    else:
        print(f"Error: Zip file not found at {ZIP_PATH}")
        sys.exit()

# --- 4. DATASET CLASS ---
class GTA5Aug1Dataset(Dataset):
    def __init__(self, root_dir):
        self.images_dir = os.path.join(root_dir, 'images')
        self.masks_dir = os.path.join(root_dir, 'labels')
        self.images = sorted(os.listdir(self.images_dir))
        self.normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.id_mapping = {
            7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
            19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
            26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18
        }

    def __len__(self): return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.images[idx])
        mask_path = os.path.join(self.masks_dir, self.images[idx])
        image = Image.open(img_path).convert('RGB').resize((1280, 720), Image.BILINEAR)
        mask = Image.open(mask_path).resize((1280, 720), Image.NEAREST)

        # AUG 1 ONLY: Random Flip
        if random.random() > 0.5:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
            mask = mask.transpose(Image.FLIP_LEFT_RIGHT)

        image = self.normalize(image)
        mask_np = np.array(mask)
        target_mask = np.full(mask_np.shape, 255, dtype=np.uint8)
        for k, v in self.id_mapping.items(): target_mask[mask_np == k] = v
        return image, torch.from_numpy(target_mask).long()

# --- 5. RESUME TRAINING ---
print(f"Resuming Training for {CHECKPOINT_NAME}...")

model = BiSeNet(num_classes=19, context_path='resnet18').to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss(ignore_index=255)

checkpoint_path = f'/content/drive/MyDrive/{CHECKPOINT_NAME}'

if os.path.exists(checkpoint_path):
    print("Loading checkpoint weights...")
    checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    if 'optimizer_state_dict' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
else:
    print("Warning: No checkpoint found. Starting from scratch.")


start_epoch = FORCED_START_EPOCH
print(f"Forcing start at Epoch {start_epoch}")

dataset = GTA5Aug1Dataset(GTA_PATH)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)


for epoch in range(start_epoch, TARGET_EPOCHS + 1):
    model.train()
    for i, (img, lbl) in enumerate(loader):
        optimizer.zero_grad()
        out = model(img.to(DEVICE))
        loss = criterion(out[0], lbl.to(DEVICE)) + 0.1 * criterion(out[1], lbl.to(DEVICE)) + 0.1 * criterion(out[2], lbl.to(DEVICE))
        loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print(f"Epoch [{epoch}/{TARGET_EPOCHS}] Step [{i}/{len(loader)}] Loss: {loss.item():.4f}")

    # Save checkpoint
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, checkpoint_path)
    print(f"Epoch {epoch} Saved.")

Setting up environment...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Resuming Training for bisenet_aug1_checkpoint.pth...
Loading checkpoint weights...
Forcing start at Epoch 48
Epoch [48/50] Step [0/313] Loss: 0.1827
Epoch [48/50] Step [50/313] Loss: 0.1520
Epoch [48/50] Step [100/313] Loss: 0.1629
Epoch [48/50] Step [150/313] Loss: 0.1508
Epoch [48/50] Step [200/313] Loss: 0.1911
Epoch [48/50] Step [250/313] Loss: 0.2324
Epoch [48/50] Step [300/313] Loss: 0.2210
Epoch 48 Saved.
Epoch [49/50] Step [0/313] Loss: 0.2024
Epoch [49/50] Step [50/313] Loss: 0.2114
Epoch [49/50] Step [100/313] Loss: 0.2160
Epoch [49/50] Step [150/313] Loss: 0.1825
Epoch [49/50] Step [200/313] Loss: 0.2142
Epoch [49/50] Step [250/313] Loss: 0.1771
Epoch [49/50] Step [300/313] Loss: 0.1652
Epoch 49 Saved.
Epoch [50/50] Step [0/313] Loss: 0.1624
Epoch [50/50] Step [50/313] Loss: 0.1752
Epoch [50/50] Step [100/313] Loss: 0.1590

In [ ]:
import torch
import os
import sys
import numpy as np
from torch.utils.data import DataLoader, Dataset
from models.bisenet.build_bisenet import BiSeNet
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as transforms

CITYSCAPES_PATH = '/content/dataset/project_data/cityscapes'
CHECKPOINT_PATH = '/content/drive/MyDrive/bisenet_aug1_checkpoint.pth'
NUM_CLASSES = 19
BATCH_SIZE = 4
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CLASSES = [
    "Road", "Sidewalk", "Building", "Wall", "Fence", "Pole",
    "Traffic Light", "Traffic Sign", "Vegetation", "Terrain", "Sky",
    "Person", "Rider", "Car", "Truck", "Bus", "Train", "Motorcycle", "Bicycle"
]

if os.path.exists('/content/models'): sys.path.append('/content/models')

class CityscapesDataset(Dataset):
    def __init__(self, root, split='val', transform=None):
        self.root = root
        self.transform = transform
        self.images_dir = os.path.join(root, 'leftImg8bit', split)
        self.masks_dir = os.path.join(root, 'gtFine', split)
        self.images = []
        self.masks = []
        if os.path.exists(self.images_dir):
            for city in sorted(os.listdir(self.images_dir)):
                img_dir_path = os.path.join(self.images_dir, city)
                mask_dir_path = os.path.join(self.masks_dir, city)
                if not os.path.isdir(img_dir_path): continue
                for file_name in sorted(os.listdir(img_dir_path)):
                    if file_name.endswith('_leftImg8bit.png'):
                        self.images.append(os.path.join(img_dir_path, file_name))
                        mask_name = file_name.replace('_leftImg8bit.png', '_gtFine_labelTrainIds.png')
                        self.masks.append(os.path.join(mask_dir_path, mask_name))

    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB').resize((1024, 512), Image.BILINEAR)
        mask = Image.open(self.masks[idx]).resize((1024, 512), Image.NEAREST)
        if self.transform: image = self.transform(image)
        return image, torch.from_numpy(np.array(mask)).long()

print(f"Evaluating: {CHECKPOINT_PATH}")
model = BiSeNet(num_classes=NUM_CLASSES, context_path='resnet18')
model.to(DEVICE)

if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model_state_dict'])
    print("Model loaded.")
else:
    print("Checkpoint not found.")
    sys.exit()

model.eval()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
dataset = CityscapesDataset(CITYSCAPES_PATH, split='val', transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

hist = np.zeros((NUM_CLASSES, NUM_CLASSES))
print("Processing...")

with torch.no_grad():
    for images, labels in tqdm(dataloader):
        images = images.to(DEVICE)
        labels = labels.numpy()
        output = model(images)
        if isinstance(output, tuple): output = output[0]
        preds = torch.argmax(output, dim=1).cpu().numpy()
        mask = (labels >= 0) & (labels < NUM_CLASSES)
        hist += np.bincount(
            NUM_CLASSES * labels[mask].astype(int) + preds[mask],
            minlength=NUM_CLASSES ** 2
        ).reshape(NUM_CLASSES, NUM_CLASSES)

iou = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
miou = np.nanmean(iou)

print(f"\nFinal mIoU: {miou * 100:.2f}%")
print("-" * 30)
for i, class_name in enumerate(CLASSES):
    print(f"{class_name:15s}: {iou[i] * 100:.2f}%")
print("-" * 30)

Evaluating: /content/drive/MyDrive/bisenet_aug1_checkpoint.pth
Model loaded.
Processing...


100%|██████████| 125/125 [00:58<00:00,  2.15it/s]


Final mIoU: 16.45%
------------------------------
Road           : 8.39%
Sidewalk       : 10.40%
Building       : 55.77%
Wall           : 13.54%
Fence          : 8.44%
Pole           : 4.36%
Traffic Light  : 5.91%
Traffic Sign   : 3.92%
Vegetation     : 70.44%
Terrain        : 5.92%
Sky            : 66.95%
Person         : 32.06%
Rider          : 0.00%
Car            : 22.15%
Truck          : 1.44%
Bus            : 2.90%
Train          : 0.00%
Motorcycle     : 0.00%
Bicycle        : 0.00%
------------------------------
